In [9]:
import dataclasses
from collections import defaultdict
from io import BytesIO, SEEK_END
from typing import Iterable, Optional

import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.io import ascii
from astropy.table import MaskedColumn, Table

from calibration.des import Des

In [10]:
DECAM_PASSBANDS = 'grizy'

@dataclasses.dataclass(kw_only=True)
class Photometry:
    name: str
    band: str
    obs: Optional[float] = None
    err: Optional[float] = None
    syn: Optional[float] = None
    residual: Optional[float] = None

    def __post_init__(self):
        if self.obs is not None and self.syn is not None and self.residual is None:
            self.residual = self.syn - self.obs


def photometry_to_table(phots: Iterable[Photometry]) -> Table:
    table = Table([dataclasses.asdict(phot) for phot in phots], masked=True)
    table.add_index(['name', 'band'])
    for column in table.columns:
        if table[column].dtype is object:
            table[column] = MaskedColumn(table[column], dtype=float, mask=[x is None for x in table[column]])
    return table


def print_statistics(phots, bands=None):
    table = photometry_to_table(phots)
    if bands is None:
        bands = np.unique(table['band'])
    for band in bands:
        idx = (~table['residual'].mask) & (table['band'] == band)
        residuals = table['residual'][idx]
        obs_magerr = table['err'][idx]
        rms = np.sqrt(np.mean(np.square(residuals)))
        wmean = np.average(residuals, weights=obs_magerr ** -2)
        # I think we would like to assume here that mean is zero, ddof=N
        chi2 = np.sum(np.square(residuals / obs_magerr))
        print(f'{band}: {rms = :.3f} {wmean = :.3f} chi2/N = {chi2:.1f}/{np.sum(idx)}')

In [11]:
def read_table(filename, columns):
    io = BytesIO(rf'''
        \begin{{table}}
        \begin{{tabular}}{{{'l'*len(columns)}}}
        {'&'.join(columns)}
    '''.encode())
    io.seek(0, SEEK_END)
    with open(filename, 'rb') as fh:
        io.write(fh.read())
    io.write(br'''
        \end{tabular}
        \end{table}
    ''')
    io.seek(0)
    table = ascii.read(io, format='latex', converters={'*': str})
    return table

In [12]:
table = read_table(
    'axelrod_tables/lco.tex',
    columns=['Star', 'Orig. name', 'RA', 'DEC', 'PM_RA', 'PM_DEC', 'G', 'Rp', 'Bp',],
)
names_coords = {name: SkyCoord(ra=ra, dec=dec, unit=['hour', 'deg'])
                for name, ra, dec in table[['Star', 'RA', 'DEC']] if name.startswith('WDFS')}
del table

# DES = DECam

In [13]:
decam_table = read_table(
    'axelrod_tables/decam.tex',
    columns=['obj', 'g', 'Sg', 'r', 'Sr', 'i', 'Si', 'z', 'Sz', 'y', 'Sy',],
)
for column in decam_table.columns:
    if not column.startswith('S'):
        continue
    decam_table[column] = np.asarray(decam_table[column], dtype=float)
decam_table.add_index('obj')

In [14]:
def extract_des(des, mag_type, magerr_type, max_flags):
    phots = []
    for name in decam_table['obj']:
        coord = names_coords[name]
        df = des.cone_search(coord, radius_arcsec=1)
        match df.shape[0]:
            case 0: continue
            case 1: row = df.iloc[0]
            case _: raise RuntimeError(f'Too many results in DES DR{dr} for {name}')
        for band in DECAM_PASSBANDS:
            obs_mag_item = row[f'{mag_type}_{band}']
            if obs_mag_item >= 90.0:  # 99 is special
                continue
            if row[f'flags_{band}'] > max_flags:
                continue
            phots.append(Photometry(
                name=name,
                band=band,
                obs=obs_mag_item,
                err=row[f'{magerr_type}_{band}'],
                syn=decam_table.loc[name][f'S{band}'],
            ))
    return phots

In [15]:
for dr in [1,2]:
    des = Des(dr=dr)
    # flag 1 means aperture photometry is affected
    for mag_type, magerr_type, max_flags in [('mag_auto', 'magerr_auto', 0),
                                            ('wavg_mag_psf', 'wavg_magerr_psf', 1)]:
        print(f'DES DR{dr} {mag_type}')
        phots = extract_des(des, mag_type, magerr_type, max_flags)
        print_statistics(phots, DECAM_PASSBANDS)
        del phots
    del des

DES DR1 mag_auto
g: rms = 0.054 wmean = -0.041 chi2/N = 6711.2/8
r: rms = 0.025 wmean = -0.017 chi2/N = 444.6/7
i: rms = 0.025 wmean = -0.019 chi2/N = 127.7/8
z: rms = 0.034 wmean = -0.008 chi2/N = 12.3/8
y: rms = 0.166 wmean = -0.071 chi2/N = 37.2/8
DES DR1 wavg_mag_psf
g: rms = 0.012 wmean = -0.011 chi2/N = 387.5/8
r: rms = 0.010 wmean = 0.005 chi2/N = 169.8/7
i: rms = 0.012 wmean = 0.005 chi2/N = 69.1/8
z: rms = 0.017 wmean = 0.017 chi2/N = 75.9/8
y: rms = 0.053 wmean = -0.008 chi2/N = 8.5/8
DES DR2 mag_auto
g: rms = 0.031 wmean = -0.028 chi2/N = 5205.2/7
r: rms = 0.020 wmean = -0.016 chi2/N = 685.4/7
i: rms = 0.020 wmean = -0.015 chi2/N = 152.3/7
z: rms = 0.029 wmean = -0.008 chi2/N = 17.4/7
y: rms = 0.137 wmean = -0.069 chi2/N = 40.5/7
DES DR2 wavg_mag_psf
g: rms = 0.014 wmean = -0.009 chi2/N = 869.2/7
r: rms = 0.005 wmean = 0.000 chi2/N = 58.7/7
i: rms = 0.005 wmean = 0.001 chi2/N = 40.6/7
z: rms = 0.014 wmean = 0.015 chi2/N = 111.8/7
y: rms = 0.035 wmean = -0.017 chi2/N = 11.6/7

In [16]:
des = Des(dr=2)
# flag 1 means aperture photometry is affected, we don't care about it for PSF
mag_type, magerr_type, max_flags = 'wavg_mag_psf', 'wavg_magerr_psf', 1
des_phots = extract_des(des, mag_type, magerr_type, max_flags)
des_phots_table = photometry_to_table(des_phots)

new_decam_table = decam_table.copy()
for band in DECAM_PASSBANDS:
    new_decam_table[band] = ''
for name in new_decam_table['obj']:
    decam_row = new_decam_table.loc[name]
    for band in DECAM_PASSBANDS:
        try:
            des_phots_row = des_phots_table[(des_phots_table['name'] == name) & (des_phots_table['band'] == band)][0]
        except IndexError:
            continue
        if des_phots_row['obs'] is None or des_phots_row['err'] is None:
            continue
        decam_row[band] = f'{des_phots_row["obs"]:.3f} ({des_phots_row["err"]*1e3:.0f})'
for column in new_decam_table.columns:
    test_value = new_decam_table[column][0]
    if not isinstance(test_value, str):
        new_decam_table[column] = [f'{x:.3f}' for x in new_decam_table[column]]
ascii.write(new_decam_table, 'axelrod_tables/new_decam.tex', format='latex')

/Users/hombit/.cenv/calibration/lib/python3.10/site-packages/astropy/table/row.py:65: StringTruncateWarning: truncated right side string(s) longer than 10 character(s) during assignment
  self._table.columns[item][self._index] = val
/Users/hombit/.cenv/calibration/lib/python3.10/site-packages/astropy/table/row.py:65: StringTruncateWarning: truncated right side string(s) longer than 10 character(s) during assignment
  self._table.columns[item][self._index] = val
/Users/hombit/.cenv/calibration/lib/python3.10/site-packages/astropy/table/row.py:65: StringTruncateWarning: truncated right side string(s) longer than 11 character(s) during assignment
  self._table.columns[item][self._index] = val
/Users/hombit/.cenv/calibration/lib/python3.10/site-packages/astropy/table/row.py:65: StringTruncateWarning: truncated right side string(s) longer than 11 character(s) during assignment
  self._table.columns[item][self._index] = val
/Users/hombit/.cenv/calibration/lib/python3.10/site-packages/astropy

## Legacy Survey

In [17]:
from calibration import legacy_survey

DECALS_PASSBANDS = 'grz'

ls = legacy_survey.LegacySurvey(dr=9)
ls_phots = []

for name, coord in names_coords.items():
    df = ls.cone_search(coord, radius_arcsec=1)
    match df.shape[0]:
        case 0: continue
        case 1: row = df.iloc[0]
        case _: raise RuntimeError(f'Too many results in DES DR{dr} for {name}')
    for band in DECALS_PASSBANDS:
        obs_mag_item = row[f'mag_{band}']
        obs_magerr_item = 2.5 / np.log(10.0) / row[f'snr_{band}']
        if obs_mag_item > 25:
            raise RuntimeError(f'{name} obs_mag_item {band} = {obs_mag_item}')
        if row[f'allmask_{band}'] != 0:
            raise RuntimeError(f"{name} allmask {band} = {row[f'allmask_{band}']}")
        # if row[f'fracmasked_{band}'] > 0.2:
        #     print(f"{name} fracmasked {band} = {row[f'fracmasked_{band}']:.2f}, {obs_mag_item = :.3f}, {obs_magerr_item = :.3f}")
        #     continue
        if not (0 < obs_magerr_item < 0.1):
            raise RuntimeError(f'{name} obs_magerr_item {band} = {obs_magerr_item}')
        ls_phots.append(Photometry(
            name=name,
            band=band,
            obs=obs_mag_item,
            err=obs_magerr_item,
            syn=decam_table.loc[name][f'S{band}'],
        ))

print_statistics(ls_phots, DECALS_PASSBANDS)

g: rms = 0.019 wmean = 0.012 chi2/N = 8364.5/21
r: rms = 0.043 wmean = 0.039 chi2/N = 17246.7/21
z: rms = 0.027 wmean = 0.027 chi2/N = 1956.3/21


In [27]:
ls_phots_table = photometry_to_table(ls_phots)
ls_table = decam_table.copy()
for band in DECAM_PASSBANDS:
    ls_table[band] = ''
for name in ls_table['obj']:
    ls_row = ls_table.loc[name]
    for band in DECALS_PASSBANDS:
        try:
            ls_phots_row = ls_phots_table[(ls_phots_table['name'] == name) & (ls_phots_table['band'] == band)][0]
        except IndexError:
            continue
        if ls_phots_row['obs'] is None or ls_phots_row['err'] is None:
            continue
        ls_row[band] = f'{ls_phots_row["obs"]:.3f} ({ls_phots_row["err"]*1e3:.0f})'
for column in ls_table.columns:
    test_value = ls_table[column][0]
    if not isinstance(test_value, str):
        ls_table[column] = [f'{x:.3f}' for x in ls_table[column]]
ascii.write(ls_table[['obj', 'g', 'Sg', 'r', 'Sr', 'z', 'Sz']], 'axelrod_tables/legacy_survey.tex', format='latex')

## Different `fracmasked_` thresholds

### 0.01
```
g: rms = 0.021 wmean = -0.012 chi2/N = 1366.3/10
r: rms = 0.044 wmean = -0.044 chi2/N = 8244.7/16
z: rms = 0.028 wmean = -0.021 chi2/N = 427.3/14
```

### 0.05
```
g: rms = 0.019 wmean = -0.010 chi2/N = 1585.7/13
r: rms = 0.043 wmean = -0.039 chi2/N = 15898.5/20
z: rms = 0.027 wmean = -0.022 chi2/N = 491.8/17
```

### 0.1
```
g: rms = 0.019 wmean = -0.009 chi2/N = 2296.0/16
r: rms = 0.043 wmean = -0.039 chi2/N = 15898.5/20
z: rms = 0.026 wmean = -0.022 chi2/N = 494.0/18
```

### 0.2
```
g: rms = 0.019 wmean = -0.012 chi2/N = 8364.5/21
r: rms = 0.043 wmean = -0.039 chi2/N = 17246.7/21
z: rms = 0.026 wmean = -0.027 chi2/N = 1899.3/20
```

### 1.0
```
g: rms = 0.019 wmean = -0.012 chi2/N = 8364.5/21
r: rms = 0.043 wmean = -0.039 chi2/N = 17246.7/21
z: rms = 0.027 wmean = -0.027 chi2/N = 1956.3/21
```

# PS1

In [19]:
ps1_table = read_table(
    'axelrod_tables/ps.tex',
    columns=['obj', 'g', 'gSynth', 'r', 'rSynth', 'i', 'iSynth', 'z', 'zSynth']
)
for column in ps1_table.columns:
    if not column.endswith('Synth'):
        continue
    ps1_table[column] = np.asarray(ps1_table[column], dtype=float)
ps1_table.add_index('obj')

In [20]:
from calibration import ps1

PS_PASSBANDS = 'griz'

In [21]:
def extract_ps1(dr, average_method, mag_type):
    mag_prefix = {'mean': 'Mean', 'stacked': ''}[average_method]
    qf_prefix = {'mean': '', 'stacked': 'psf'}[average_method]
    ps = ps1.Ps1(dr=dr)
    method = getattr(ps, f'{average_method}_objects')
    phots = []
    for name in ps1_table['obj']:
        coord = names_coords[name]
        df = method(coord).to_pandas()
        match df.shape[0]:
            case 0: continue
            case 1: row = df.iloc[0]
            case _: raise RuntimeError(f'Too many results in PS1 DR{dr} {average_method} for {name}')
        for band in PS_PASSBANDS:
            mag_column = f'{band}{mag_prefix}{mag_type}Mag'
            err_column = f'{mag_column}Err'
            qf_perfect_column = f'{band}{qf_prefix}QfPerfect'
            obs_mag_item = row[mag_column]
            obs_err_item = row[err_column]
            qf_perfec_item = row[qf_perfect_column]
            if qf_perfec_item < 0.9:
                continue
            phots.append(Photometry(
                name=name,
                band=band,
                obs=obs_mag_item,
                err=obs_err_item,
                syn=ps1_table.loc[name][f'{band}Synth'],
            ))
    return phots

In [22]:
for dr in [2]:
    for average_method in ['mean', 'stacked']:
        for mag_type in ['Ap', 'PSF']:
            print(f'PS1 DR{dr} {average_method} {mag_type}')
            ps_phots = extract_ps1(dr, average_method, mag_type)
            print_statistics(ps_phots, PS_PASSBANDS)
            del ps_phots

PS1 DR2 mean Ap
g: rms = 0.020 wmean = 0.003 chi2/N = 190.7/23
r: rms = 0.044 wmean = -0.014 chi2/N = 224.3/23
i: rms = 0.059 wmean = -0.027 chi2/N = 467.3/23
z: rms = 0.098 wmean = -0.085 chi2/N = 932.9/23
PS1 DR2 mean PSF
g: rms = 0.019 wmean = -0.014 chi2/N = 326.9/23
r: rms = 0.019 wmean = -0.017 chi2/N = 155.1/23
i: rms = 0.034 wmean = -0.029 chi2/N = 719.3/23
z: rms = 0.076 wmean = -0.078 chi2/N = 1428.0/23
PS1 DR2 stacked Ap
g: rms = 0.024 wmean = -0.015 chi2/N = 8265.1/23
r: rms = 0.044 wmean = -0.017 chi2/N = 10784.6/23
i: rms = 0.057 wmean = -0.034 chi2/N = 24622.8/23
z: rms = 0.118 wmean = -0.080 chi2/N = 40095.5/23
PS1 DR2 stacked PSF
g: rms = 0.082 wmean = -0.049 chi2/N = 16953.3/23
r: rms = 0.095 wmean = -0.047 chi2/N = 13399.3/23
i: rms = 0.090 wmean = -0.027 chi2/N = 8343.4/23
z: rms = 0.129 wmean = -0.069 chi2/N = 3307.6/23


In [25]:
ps = ps1.Ps1(dr=2)
dr, average_method, mag_type = 2, 'mean', 'Ap'
ps_phots = extract_ps1(dr, average_method, mag_type)
ps_phots_table = photometry_to_table(ps_phots)

new_ps1_table = ps1_table.copy()
for band in PS_PASSBANDS:
    new_ps1_table[band] = ''
for name in new_ps1_table['obj']:
    ps1_row = new_ps1_table.loc[name]
    for band in PS_PASSBANDS:
        try:
            ps_phots_row = ps_phots_table[(ps_phots_table['name'] == name) & (ps_phots_table['band'] == band)][0]
        except IndexError:
            continue
        if ps_phots_row['obs'] is None or ps_phots_row['err'] is None:
            continue
        ps1_row[band] = f'{ps_phots_row["obs"]:.3f} ({ps_phots_row["err"]*1e3:.0f})'
for column in new_ps1_table.columns:
    test_value = new_ps1_table[column][0]
    if not isinstance(test_value, str):
        new_ps1_table[column] = [f'{x:.3f}' for x in new_ps1_table[column]]
ascii.write(new_ps1_table, 'axelrod_tables/new_ps.tex', format='latex')

/Users/hombit/.cenv/calibration/lib/python3.10/site-packages/astropy/table/row.py:65: StringTruncateWarning: truncated right side string(s) longer than 11 character(s) during assignment
  self._table.columns[item][self._index] = val
/Users/hombit/.cenv/calibration/lib/python3.10/site-packages/astropy/table/row.py:65: StringTruncateWarning: truncated right side string(s) longer than 11 character(s) during assignment
  self._table.columns[item][self._index] = val
/Users/hombit/.cenv/calibration/lib/python3.10/site-packages/astropy/table/row.py:65: StringTruncateWarning: truncated right side string(s) longer than 12 character(s) during assignment
  self._table.columns[item][self._index] = val
/Users/hombit/.cenv/calibration/lib/python3.10/site-packages/astropy/table/row.py:65: StringTruncateWarning: truncated right side string(s) longer than 12 character(s) during assignment
  self._table.columns[item][self._index] = val
/Users/hombit/.cenv/calibration/lib/python3.10/site-packages/astropy

OSError: File axelrod_tables/new_ps.tex already exists. If you mean to replace it then use the argument "overwrite=True".